In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import pickle
import os
from math import log
import matplotlib.pyplot as plt

from datetime import datetime

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\AAM\False Rejects")

In [3]:
df=pd.read_csv(os.getcwd()+"\Data Files\OP180NVHMaint_20202707.csv")

In [4]:
df.head(5)

,SITENAME,LINENAME,STATIONNAME,PARTNUMBER,STATUS,TOPLEVELSERIALNUMBER,PROD_TIMESTAMP,COLLID,CHARDESC,CHARVALUE
0,GMCFULL,T1XX Front Axle FA HiV,OP180B,40217205,A,01A200240722,2020-01-27 00:00:23.453,-1277705445,NVH Torque Sweep Test Cycle Count,1.0
1,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217206,A,01A200250294,2020-01-27 10:18:28.180,-1277496047,NVH Torque Sweep Test Cycle Count,1.0
2,GMCFULL,T1XX Front Axle FA HiV,OP180B,40217205,A,01A200240856,2020-01-27 05:16:01.673,-1277603951,NVH Torque Sweep Test Cycle Count,1.0
3,GMCFULL,T1XX Front Axle FA HiV,OP180B,40217206,A,01A200250299,2020-01-27 11:00:14.180,-1277482133,NVH Torque Sweep Test Cycle Count,1.0
4,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217206,A,01C200250128,2020-01-27 10:58:28.993,-1277482173,NVH Torque Sweep Test Cycle Count,1.0


In [5]:
df["PROD_TIMESTAMP"] = pd.to_datetime(df["PROD_TIMESTAMP"],format = '%Y-%m-%d %H:%M:%S.%f')

In [6]:
df_trans=pd.pivot_table(df, values='CHARVALUE', index=["SITENAME","LINENAME","STATIONNAME","PARTNUMBER","STATUS","TOPLEVELSERIALNUMBER","PROD_TIMESTAMP","COLLID"],
                    columns=['CHARDESC'], aggfunc=np.mean).reset_index()

In [7]:
df_trans["FALSE_REJECT"] = np.where((df_trans["STATUS"] == 'A')& (df_trans["NVH Torque Sweep Test Cycle Count"]>1), 1,0)

In [8]:
df_trans.head()

CHARDESC,SITENAME,LINENAME,STATIONNAME,PARTNUMBER,STATUS,TOPLEVELSERIALNUMBER,PROD_TIMESTAMP,COLLID,LH Slide To Full Depth Posn,NVH Torque Sweep Test Cycle Count,Pinion Slide To Full Depth Posn,RH Slide To Full Depth Posn,FALSE_REJECT
0,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,008200140669,2020-01-15 17:31:23.603,-1282850156,NaN,1.0,NaN,NaN,0
1,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,008200140670,2020-01-15 21:43:05.753,-1282789573,NaN,1.0,NaN,NaN,0
2,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,008200140676,2020-01-15 22:39:11.077,-1282778147,NaN,1.0,NaN,NaN,0
3,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,008200290567,2020-02-01 13:45:50.287,-1275157638,NaN,1.0,NaN,NaN,0
4,GMCFULL,T1XX Front Axle FA HiV,OP180A,40217205,A,008200400193,2020-02-13 03:09:37.750,-1270437207,NaN,1.0,NaN,NaN,0


In [9]:
df_trans.dropna(inplace = True)

In [10]:
df_trans["FALSE_REJECT"].value_counts()

0    93125
1      326
Name: FALSE_REJECT, dtype: int64

In [11]:
df_trans.to_csv(os.getcwd()+"\Data Files\Transformed_NVH_DATA.csv")
df_trans.to_pickle(os.getcwd()+"\Data Files\Transformed_NVH_DATA.pkl")